In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
cellphones = pd.read_csv(r'D:\Code\DS108\crawling\cellphones\raw_data.csv', encoding='utf-8-sig')
tgdd = pd.read_csv(r'D:\Code\DS108\crawling\tgdd\raw_data.csv', encoding='utf-8-sig')

In [3]:
data = pd.concat([cellphones, tgdd], ignore_index=True)

# Xử lí dữ liệu trùng lắp

In [4]:
# Kiểm tra duplicate trong dữ liệu
data.duplicated().sum()

1292

Một số máy có thông số giống nhau hoàn toàn -> Drop.

In [5]:
# Drop các dòng bị duplicated trong dữ liệu
data.drop_duplicates(inplace=True)

# Xử lí các dòng rỗng hoàn toàn

In [6]:
# Tìm các dòng rỗng hoàn toàn (tất cả các cột đều là NaN)
empty_rows = data.isnull().all(axis=1)
print(empty_rows.sum())

1


In [7]:
# Drop các dòng rỗng hoàn toàn trong dữ liệu
data.drop(index=data[pd.Series(empty_rows).values].index, inplace=True)

# Xử lí cột name

In [8]:
print(data['name'].unique())

['iPhone 14 Pro Max 128GB' 'iPhone 13 Pro Max 128GB'
 'OPPO Find N3 Flip 12GB 256GB' 'Samsung Galaxy S24 Plus 12GB 256GB'
 'Samsung Galaxy S24 Ultra 12GB 256GB' 'iPhone 14 Pro Max 256GB'
 'iPhone 14 Pro 128GB' 'iPhone 13 Pro Max 256GB' 'iPhone 15 Plus 128GB'
 'Samsung Galaxy S23 Ultra 8GB 256GB' 'OPPO A78 8GB 256GB'
 'OPPO Find N3 16GB 512GB' 'iPhone 15 Pro 128GB' 'iPhone 15 Pro Max 256GB'
 'Samsung Galaxy S23 8GB 128GB'
 'iPhone 16 Plus 128GB Chính hãng VN/A (3N350)' 'iPhone 11 64GB'
 'iPhone 8 Plus 64GB cũ đẹp' 'iPhone 15 Pro 256GB'
 'Samsung Galaxy S22 Ultra (12GB' 'iPhone XS Max 256GB Cũ đẹp'
 'iPhone 12 Pro Max 128GB' 'Samsung Galaxy Z Fold5 12GB 512GB'
 'Apple iPhone 7 Plus 128GB' 'Samsung Galaxy M55 (12GB 256GB)'
 'iPhone 14 128GB' 'iPhone XS Max 64GB Cũ trầy xước' 'iPhone 11 128GB'
 'iPhone 16 Pro Max 256GB | Chính hãng VN/A' 'iPhone 15 128GB'
 'iPhone 14 Plus 128GB' 'iPhone 13 256GB'
 'Samsung Galaxy A35 5G 8GB 128GB' 'Xiaomi 13T Pro 5G (12GB'
 'Samsung Galaxy Z Flip5 512GB' '

Cần loại bỏ các thông tin không cần thiết

In [9]:
data['name'] = data['name'].str.replace(r'Điện thoại ', '', regex=True).str.strip()

data = data.drop(data[data['name'] == 'false'].index)

pattern_to_remove = r'\s*(?:' \
                    r'\d+\s*GB' \
                    r'|\d+\s*RAM' \
                    r'|\d+\s*ROM' \
                    r'|\d+GB:\d+GB' \
                    r'|\d+:\d+' \
                    r'|Cũ' \
                    r'|\d+\s*TB' \
                    r'|Đã' \
                    r'|\b\d{4}\b' \
                    r'|2\s*sim' \
                    r'|\(' \
                    r'|Chính' \
                    r').*$'

data['name'] = data['name'].str.replace(pattern_to_remove, '', regex=True, flags=re.IGNORECASE)
data['name'] = data['name'].str.strip()

normalization_map = {
    'samsung': 'Samsung',
    'honor': 'HONOR',
    'mobell': 'Mobell',
    'xiaomi': 'Xiaomi',
    'iphone': 'iPhone',
    'oppo': 'OPPO',
    'nokia': 'Nokia',
    'masstel': 'Masstel',
    'realme': 'realme',
    'vivo': 'vivo',
    'itel': 'Itel',
    'tecno': 'Tecno',
    'benco': 'BENCO',
    'oneplus': 'OnePlus',
    'find': 'Find',
    'spark': 'Spark',
    'poco': 'Poco',
    'camon': 'Camon',
    'pova': 'Pova'
}

updated_names = []
for index, row in data.iterrows():
    product_name = row['name']
    
    if pd.isna(product_name):
        updated_names.append(np.nan)
        continue

    product_name_lower = str(product_name).lower()
    
    found_and_replaced = False
    for lower, normalized in normalization_map.items():
        pattern = r'\b' + re.escape(lower) + r'\b'
        
        if re.search(pattern, product_name_lower):
            product_name = re.sub(pattern, normalized, product_name, flags=re.IGNORECASE)
            found_and_replaced = True 
    
    updated_names.append(product_name)
    
data['name'] = updated_names
data['name'] = data['name'].str.replace(r'\bApple\s*', '', regex=True, flags=re.IGNORECASE)
data['name'] = data['name'].str.strip()

# Xử lí missing values

In [10]:
data.isnull().sum()

name                     0
brand                  300
color                   18
condition               64
price_old              594
price_new              620
image                    0
warranty               668
CPU                    152
RAM                    303
capacity                25
time                   874
battery                  9
screen_size             11
operating_system      2074
display_technology     778
screen_resolution      203
SIM                    114
size                   154
weight                1797
bluetooth              343
refresh_rate          2376
GPU                   1972
dtype: int64

Thuộc tính price_old là thuộc tính rất quan trọng, được xem là target của bài toán -> Drop các dòng bị thiếu thuộc tính price_old

In [11]:
# Loại bỏ các dòng bị thiếu giá trị trong cột 'price_old'
data = data.dropna(subset=['price_old'])

Điền giá trị thiếu dựa trên các giá trị có sẵn 

In [23]:
for name_value, group in data.groupby('name'):
    total = len(group)
    columns_to_check = ["CPU", "RAM", "capacity", "time", "battery", "screen_size", "operating_system", 
                    "display_technology", "screen_resolution", "SIM", "size", "weight", "bluetooth", 
                    "refresh_rate", "GPU"]
    
    for column in columns_to_check:
        missing = group[column].isna().sum()
        ratio = missing / total

        if 0 < ratio < 1:
            # Lấy giá trị đầu tiên không bị thiếu trong nhóm
            value = group[column].dropna().iloc[0]
            # Gán lại cho các dòng bị thiếu trong nhóm đó
            data.loc[(data['name'] == name_value) & (data[column].isna()), column] = value

Điền thiếu bằng cách tra các thông tin thiếu trên PhoneArena

In [13]:
# Chuyển đổi cột 'weight' sang kiểu str để điền thiếu
data['weight'] = data['weight'].astype(str)

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import time

def scrape_from_phonearena(driver, name, keywords):
    # Bấm vào icon tìm kiếm
    search_icon = driver.find_element(By.CSS_SELECTOR, ".ic.ic-search.ic-24")
    search_icon.click()

    # Nhập từ khóa vào ô tìm kiếm
    search_box = driver.find_element(By.XPATH, "//input[@class = 'input input-search']")
    search_box.send_keys(name)
    search_box.send_keys(Keys.RETURN)

    # Bấm vào link đầu tiên trong danh sách kết quả
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".title a")))
    for attempt in range(3):
        try:
            # Tìm phần tử lại trước mỗi lần thử
            first_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".title a")))
            driver.execute_script("arguments[0].scrollIntoView();", first_link)
            time.sleep(5)
            first_link.click()
            break  # Nếu click thành công, thoát vòng lặp
        except StaleElementReferenceException:
            print(f"Retrying click due to stale element (attempt {attempt + 1})...")
        except Exception as e:
            print(f"Unexpected error: {e}")  # Bắt lỗi khác nếu có
    else:
        raise Exception("Failed to click first search result after 3 attempts due to StaleElementReferenceException")

        
    result = [None] * len(keywords)
    info_labels = {
        "CPU": "Systems chip", 
        "RAM": "RAM", 
        "capacity": "Internal storage", 
        "time": "",
        "battery": "Capacity", 
        "screen_size": "Dimensions", 
        "operating_system": "OS", 
        "display_technology": "Technology", 
        "screen_resolution": "Resolution", 
        "SIM": "SIM type", 
        "size": "Size", 
        "weight": "Weight", 
        "bluetooth": "Bluetooth", 
        "refresh_rate": "Refresh rate", 
        "GPU": "GPU"
    }
    
    for idx, keyword in enumerate(keywords):
        try:
            label = info_labels.get(keyword)
            if not label and keyword != 'time':
                print(f"Không tìm thấy nhãn cho keyword: {keyword}")
                continue

            if keyword == 'time':
                xpath = "//span[contains(text(), 'Released')]//following-sibling::span[2]"
            else:
                xpath = f"//th[contains(text(), '{label}')]//following-sibling::td"

            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, xpath))
            )
            result[idx] = element.text
        except:
            print(f"Lỗi khi lấy thông tin '{keyword}' cho '{name}'")
        
    return result

In [15]:
from selenium import webdriver
import time

driver = webdriver.Chrome()
driver.get("https://www.phonearena.com/")
time.sleep(30)

# Cột cần kiểm tra
columns_to_check = ["CPU", "RAM", "capacity", "time", "battery", "screen_size", "operating_system", 
                    "display_technology", "screen_resolution", "SIM", "size", "weight", "bluetooth", 
                    "refresh_rate", "GPU"]

for phone in data['name'].unique():
    mask = (data['name'] == phone)  # Lọc tất cả các dòng có cùng `name`
    missing_columns = [col for col in columns_to_check if data.loc[mask, col].isnull().any()]  

    if missing_columns:
        try:
            print(f"Scraping {phone} cho các thuộc tính bị thiếu: {missing_columns}")
            specs = scrape_from_phonearena(driver, phone, missing_columns)

            # Điền dữ liệu vào tất cả các dòng có cùng `name`
            for i, col in enumerate(missing_columns):
                if specs[i] is not None:
                    data.loc[mask & data[col].isnull(), col] = specs[i]
        except:
            pass

driver.quit()



Scraping Samsung Galaxy S23 Ultra cho các thuộc tính bị thiếu: ['time', 'refresh_rate']
Scraping Samsung Galaxy S23 cho các thuộc tính bị thiếu: ['time', 'operating_system', 'refresh_rate']
Scraping iPhone 8 Plus cho các thuộc tính bị thiếu: ['time', 'refresh_rate']
Lỗi khi lấy thông tin 'refresh_rate' cho 'iPhone 8 Plus'
Scraping Samsung Galaxy S22 Ultra cho các thuộc tính bị thiếu: ['time']
Scraping iPhone XS Max cho các thuộc tính bị thiếu: ['RAM', 'time', 'refresh_rate']
Lỗi khi lấy thông tin 'refresh_rate' cho 'iPhone XS Max'
Scraping iPhone 12 Pro Max cho các thuộc tính bị thiếu: ['time']
Scraping iPhone 7 Plus cho các thuộc tính bị thiếu: ['RAM', 'time', 'display_technology', 'refresh_rate']
Lỗi khi lấy thông tin 'refresh_rate' cho 'iPhone 7 Plus'
Scraping iPhone 12 Pro cho các thuộc tính bị thiếu: ['RAM', 'time', 'display_technology', 'refresh_rate']
Scraping vivo Y18S cho các thuộc tính bị thiếu: ['time', 'refresh_rate']
Lỗi khi lấy thông tin 'time' cho 'vivo Y18S'
Lỗi khi lấy

Lưu dữ liệu sau điền thiếu

In [25]:
# Xuất DataFrame ra file CSV
data.to_csv('updated_data.csv', index=False, encoding='utf-8-sig')